In [ ]:
!pip install flask psycopg2-binary geopandas folium




In [1]:
from flask import Flask, jsonify, render_template
import psycopg2
from psycopg2.extras import RealDictCursor
import geopandas as gpd
import folium
from config import db_name, user, password, host

app = Flask(__name__)
app.config['DEBUG'] = True

def get_db_connection():
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=user,
            password=password,
            host=host
        )
        return conn
    except psycopg2.Error as e:
        app.logger.error(f"Error connecting to database: {e}")
        return None

@app.route('/landslides/surface', methods=['GET'])
def get_landslides():
    conn = get_db_connection()
    if conn is None:
        return jsonify({"error": "Database connection error"}), 500
    
    try:
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute("SELECT ST_AsText(geometry) AS geometry_wkt, ar_kmq , nome, ar_fr_p2, ar_fr_p1, ar_fr_p3p4, ar_frp3p4p FROM dataset")
        landslides = cursor.fetchall()
        cursor.close()
        conn.close()
        return jsonify(landslides)
    except psycopg2.Error as e:
        app.logger.error(f"Error executing SQL query: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

@app.route('/map')
def generate_map():
    conn = get_db_connection()
    if conn is None:
        return "Error connecting to the database", 500

    try:
        # Query to fetch data
        query = """
        SELECT
            cod_reg,
            cod_rip,
            COD_PROV,
            ar_kmq,
            nome,
            uid,
            ar_fr_p3p4,
            ar_fr_p2,
            ar_fr_p1,
            pop_fr_p2,
            pop_fr_p1,
            ed_fr_p2,
            ed_fr_p1,
            ar_frp3p4p,
            popfrp3p4p,
            ed_fr_p3p4,
            edfrp3p4p,
            ST_AsText(geometry) as geometry
        FROM dataset
        """
        df = gpd.read_postgis(query, conn)
        conn.close()

        # Convert 'geometry' column to GeoSeries
        df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
        gdf = gpd.GeoDataFrame(df, geometry='geometry')

        # Create a Folium map centered on a specific location
        m = folium.Map(location=[42.5, 12.5], zoom_start=6)

        # Add a choropleth layer to the map
        folium.Choropleth(
            geo_data=gdf,
            name='choropleth',
            data=gdf,
            columns=['uid', 'ar_fr_p3p4'],
            key_on='feature.properties.uid',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Surface area with very high and very high landslide risk (km2)'
        ).add_to(m)

        # Save the map as an HTML file
        m.save('templates/map.html')
        return render_template('map.html')

    except Exception as e:
        app.logger.error(f"Error processing data: {e}")
        return "Error processing data", 500

if __name__ == '__main__':
    app.run(debug=True, port=8000)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\91700\anaconda3\envs\se4g24\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
